In [6]:
!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [17]:
df=pd.read_csv('./AmazonDataset/Label10000.csv')
print(len(df))
# df = df[df['text'].str.split().str.len().gt(5)]
# df = df[df['text'].str.split().str.len().lt(30)]
df=df.reset_index(drop=None)
print(len(df))
# news=df['text']
# label=df['label']
# len(news)
# current_words=[]
# for i in range(len(news)):
#   # for word in news.loc[i].split(" "):
#     current_words.append(news.loc[i])
# print((current_words[0]))

10000
10000


In [18]:
df['word_count']=df['text'].apply(lambda x: len(str(x).split(" ")))
df.word_count.describe()
common_words=pd.Series(''.join(df['text']).split()).value_counts()
print(common_words[:10])
print("Not Common words")
print(common_words[-30000:])

appl      5971
aapl      2096
inc       2077
share     1790
nasdaq    1219
buy        978
iphon      973
rate       963
report     949
price      917
dtype: int64
Not Common words
appl           5971
aapl           2096
inc            2077
share          1790
nasdaq         1219
               ... 
yearsellsid       1
larson            1
reportlet         1
commonli          1
aroundappl        1
Length: 13214, dtype: int64


In [21]:
from nltk.tokenize import word_tokenize
most_common=common_words[:3]
least_common=common_words[-12000:]
texts=[]
labels=[]
def removeWords(words):
    final_words=[]
    for word in words:
        if not (word in most_common or word in least_common):
            final_words.append(word)
    return final_words
for i in range(len(df)):
    if i%10000==0:
        print("------>",i)
    words=word_tokenize(df.loc[i].text)
    # print(df.loc[i].text)
    words=removeWords(words)
    extracted_sentence=' '.join(word for word in words)
    # print(df.loc[i].text)
    texts.append(extracted_sentence)
    labels.append(df.loc[i]['label'])
new_df=pd.DataFrame({
    'text':texts,
    'label':labels
})


------> 0


In [22]:
new_df.tail()

,text,label
9995,immedi respond request comment friday,1
9996,great news iphon se fan,1
9997,earli nasdaq introduc smartphon known iphon se,1
9998,nearli two year launch iphon se still nt updat...,1
9999,main compani tri boost sale iphon se price eve...,1


In [23]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [24]:
news=new_df['text']
label=new_df['label']
tokenized = news.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))


In [25]:
news

0       also featur annual tom speaker new job profess...
1       huge anticip iphon x samsung follow note big y...
2            iphon x best phone money issu one expens buy
3       premium devic start per month depend purchas p...
4       face id technolog camera help make iphon need ...
                              ...                        
9995                immedi respond request comment friday
9996                              great news iphon se fan
9997       earli nasdaq introduc smartphon known iphon se
9998    nearli two year launch iphon se still nt updat...
9999    main compani tri boost sale iphon se price eve...
Name: text, Length: 10000, dtype: object

In [26]:
tokenized.head()

0    [101, 2036, 8658, 3126, 3296, 3419, 5882, 2047...
1    [101, 4121, 3424, 6895, 2361, 12997, 8747, 106...
2    [101, 12997, 8747, 1060, 2190, 3042, 2769, 263...
3    [101, 12882, 14386, 2278, 2707, 2566, 3204, 12...
4    [101, 2227, 8909, 21416, 21197, 4950, 2393, 21...
Name: text, dtype: object

In [27]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [28]:
padded

array([[  101,  2036,  8658, ...,     0,     0,     0],
       [  101,  4121,  3424, ...,     0,     0,     0],
       [  101, 12997,  8747, ...,     0,     0,     0],
       ...,
       [  101,  4656,  2072, ...,     0,     0,     0],
       [  101,  2379,  3669, ...,     0,     0,     0],
       [  101,  2364,  4012, ...,     0,     0,     0]])

In [29]:
np.array(padded).shape

(10000, 238)

In [30]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10000, 238)

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, label)

In [ ]:
train_features